<a href="https://colab.research.google.com/github/s34836/BIG_DATA/blob/main/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In Spark (either SQL or not) create a script to find the most obscure Marvel superheros based on two files you can find in Class Materials.

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, size, col
spark = SparkSession.builder.appName("ObscureMarvelHero").getOrCreate()

# Marvel graph
graph_raw = spark.read.text("MarvelGraph.txt")
graph_raw.createOrReplaceTempView("graph_raw")
# Count connections
graph_df = spark.sql("SELECT CAST(SPLIT(value, ' ')[0] AS INT) as hero_id, \
                            (SIZE(SPLIT(value, ' ')) - 1) - 1 as connection_count \
                     FROM graph_raw")

# Marvel names
names_raw = spark.read.text("MarvelNames.txt")
names_raw.createOrReplaceTempView("names_raw")
names_df = spark.sql("SELECT CAST(SPLIT(value, ' ')[0] AS INT) as hero_id, \
                            SPLIT(value, ' ', 2)[1] as hero_name \
                     FROM names_raw")

# Join graph with names
hero_df = graph_df.join(names_df, "hero_id", "inner")
hero_df.createOrReplaceTempView("heroes")

# Find minimum connection count
min_connections = spark.sql("SELECT MIN(connection_count) as min_connections FROM heroes")
min_connections.show()

# Count heroes with minimum connections
count_min = spark.sql("SELECT COUNT(*) as count_of_heroes_with_min_connections \
                      FROM heroes \
                      WHERE connection_count = (SELECT MIN(connection_count) FROM heroes)")
count_min.show()

# Find most obscure heroes (those with minimum connections)
result = spark.sql("SELECT hero_id, hero_name, connection_count \
                   FROM heroes \
                   WHERE connection_count = (SELECT MIN(connection_count) FROM heroes) \
                   ORDER BY hero_name ASC")
result.show(truncate=False)
spark.stop()

+---------------+
|min_connections|
+---------------+
|              0|
+---------------+

+------------------------------------+
|count_of_heroes_with_min_connections|
+------------------------------------+
|                                  19|
+------------------------------------+

+-------+----------------------+----------------+
|hero_id|hero_name             |connection_count|
+-------+----------------------+----------------+
|467    |"BERSERKER II"        |0               |
|577    |"BLARE/"              |0               |
|835    |"CALLAHAN, DANNY"     |0               |
|1089   |"CLUMSY FOULUP"       |0               |
|1408   |"DEATHCHARGE"         |0               |
|1841   |"FENRIS"              |0               |
|2117   |"GERVASE, LADY ALYSSA"|0               |
|2139   |"GIURESCU, RADU"      |0               |
|3014   |"JOHNSON, LYNDON BAIN"|0               |
|2911   |"KULL"                |0               |
|3298   |"LUNATIK II"          |0               |
|3490   |"MAR